In [1]:
import snowflake.connector
import yaml
import src.policy_helper as policy_helper
import time

In [2]:
query_config = yaml.safe_load(open('../config.yaml', 'r'))


In [3]:
release_config = yaml.safe_load(open('../expected_config.yaml', 'r'))

In [4]:
con = snowflake.connector.connect(
    user=query_config['user'],
    authenticator=query_config['authenticator'],
    account=query_config['account'],
    warehouse=query_config['warehouse'],
    database=query_config['database'],
    session_parameters={
        'QUERY_TAG': 'Get ODD ID by name'
    }
)

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://accounts.google.com/o/saml2/idp?idpid=C01dz3puk&SAMLRequest=nZJNb%2BIwEIb%2FSuQ9J3ZCpAaLgFhQtZG6CwtppfZmkoF6cezUH6Tw69fhQ%2Boe2sMeLFn2O%2FPMzDujyXsjggNow5XMURwRFICsVM3lLkeP5X2YocBYJmsmlIQcHcGgyXhkWCNaOnX2Va7gzYGxgU8kDe0%2FcuS0pIoZbqhkDRhqK7qe%2FnygSUQoMwa09Th0DakN96xXa1uKcdd1UTeIlN7hhBCCyRB7VS%2F5hj4g2q8ZrVZWVUrcQt59T58gYkzSHuEVnrC8Bn7n8jKCryibi8jQH2W5DJeLdYmC6a27mZLGNaDXoA%2B8gsfVw6UA4ytYlMNhliQRuLACaTUTYRwZqbqtYHuoVNM66xNH%2Foa3UGOhdtyPq5jnqN3zulhbkhm4e8uGjSsPi3RVxU9%2FFqnlcXE8HZtnUOR0Wmx%2Bb9IKBU83c5Pe3MIYB4XsLbX%2BiSSDME7COC3JkMYpTbPoLhu8oGDuLeWS2XPkrW5WVcpJa6KdUjsB5%2FoU7v1IMK%2FbiT%2B8zmckrk%2BD1u3RZU%2FomanH%2F9H9CH9McN26X96IYr5UglfH4F7phtnPfYqj%2BPzC63B7llJoGBfTutZgjPdLCNXNNDDrl9tqBwiPL9R%2F13v8Fw%3D%3D&RelayState=58641 to au

In [5]:
con

In [46]:
vehicle_query = """
select
  TO_CHAR(_ingested_at, 'YYYY-MM-DD HH24:MI') AS "Session started UTC"
  , vehicle:name as "name"
  , concat(vehicle:depb_left_version:major, '.', vehicle:depb_left_version:minor, '.', vehicle:depb_left_version:patch) as "DEPB left"
  , concat(vehicle:depb_right_version:major, '.', vehicle:depb_right_version:minor, '.', vehicle:depb_right_version:patch) as "DEPB right"
  , concat(vehicle:software_version:sem_ver_tag:major, '.', vehicle:software_version:sem_ver_tag:minor, '.', vehicle:software_version:sem_ver_tag:patch) as "Vehicle vDrive"
  , concat(vehicle:reecu:mcu_fw_version:major, '.', vehicle:reecu:mcu_fw_version:minor, '.', vehicle:reecu:mcu_fw_version:patch) as "Vehicle REECU"
  , concat(vehicle:reecu:sec_fpga_gw_version:major, '.', vehicle:reecu:sec_fpga_gw_version:minor, '.', vehicle:reecu:sec_fpga_gw_version:patch) as "Vehicle SEC"
from raw_driving.session_init
where true
  and vehicle:name LIKE any ('ve-0%', 've-us-%')
  and _ingested_at >= CURRENT_DATE() - INTERVAL '30 DAY'
  and concat(vehicle:depb_right_version:major, '.', vehicle:depb_right_version:minor, '.', vehicle:depb_right_version:patch) is NOT NULL
  and concat(vehicle:reecu:mcu_fw_version:major, '.', vehicle:reecu:mcu_fw_version:minor, '.', vehicle:reecu:mcu_fw_version:patch) is NOT NULL
  and concat(vehicle:software_version:sem_ver_tag:major, '.', vehicle:software_version:sem_ver_tag:minor, '.', vehicle:software_version:sem_ver_tag:patch) is NOT NULL
  and concat(vehicle:reecu:sec_fpga_gw_version:major, '.', vehicle:reecu:sec_fpga_gw_version:minor, '.', vehicle:reecu:sec_fpga_gw_version:patch) is NOT NULL
  QUALIFY 
  ROW_NUMBER() OVER (PARTITION BY vehicle:name ORDER BY _ingested_at DESC) = 1
  ORDER BY vehicle:name ASC
LIMIT 50
"""

telestaion_query = """
select
  TO_CHAR(_ingested_at, 'YYYY-MM-DD HH24:MI') AS "Session started UTC"
  , telestation:name as "name"
  , concat(telestation:software_version:sem_ver_tag:major, '.', telestation:software_version:sem_ver_tag:minor, '.', telestation:software_version:sem_ver_tag:patch) as "Telestation vDrive"
  , concat(telestation:reecu:mcu_fw_version:major, '.', telestation:reecu:mcu_fw_version:minor, '.', telestation:reecu:mcu_fw_version:patch) as "Telestation REECU"
  , concat(telestation:reecu:sec_fpga_gw_version:major, '.', telestation:reecu:sec_fpga_gw_version:minor, '.', telestation:reecu:sec_fpga_gw_version:patch) as "Telestation SEC"
from raw_driving.session_init
where true
  and telestation:name LIKE any ('ts-ber-0%', 'ts-las-0%', 'ts-gd%', 'ts-ham-0%')
  and _ingested_at >= CURRENT_DATE() - INTERVAL '30 DAY'
  and concat(telestation:reecu:mcu_fw_version:major, '.', telestation:reecu:mcu_fw_version:minor, '.', telestation:reecu:mcu_fw_version:patch) is NOT NULL
  and concat(telestation:software_version:sem_ver_tag:major, '.', telestation:software_version:sem_ver_tag:minor, '.', telestation:software_version:sem_ver_tag:patch) is NOT NULL
  and concat(telestation:reecu:sec_fpga_gw_version:major, '.', telestation:reecu:sec_fpga_gw_version:minor, '.', telestation:reecu:sec_fpga_gw_version:patch) is NOT NULL
  QUALIFY 
  ROW_NUMBER() OVER (PARTITION BY telestation:name ORDER BY _ingested_at DESC) = 1
  ORDER BY vehicle:name ASC
LIMIT 50
"""


In [47]:
vehicle_output = con.cursor().execute(vehicle_query).fetchall()

In [48]:
telestation_output = con.cursor().execute(telestaion_query).fetchall()

In [23]:
vehicle_configuration = {}

for vehicle in vehicle_output:
    vehicle_configuration[vehicle[1].replace('"', '')] = {
        'depb_right': vehicle[2],
        'depb_left': vehicle[3],
        'vehicle_vdrive': vehicle[4],
        'vehicle_reecu': vehicle[5],
        'vehicle_sec': vehicle[6],
        'date': vehicle[0]
    }

telestation_configuration = {}
for telestation in telestation_output:
    telestation_configuration[telestation[1].replace('"', '')] = {
        'telestation_vdrive': telestation[2],
        'telestation_reecu': telestation[3],
        'telestation_sec': telestation[4]
    }


In [49]:
mismatched_vehicles = {}
matched_vehicles = {}
vsr_keys = ["vdrive", "reecu", "vehicle_sec", "depb"]
vehicle_keys = ["vehicle_vdrive", "vehicle_reecu", "vehicle_sec", "depb_left"]
for vsr_config in release_config:
    mismatched_vehicles[vsr_config["VSR"]] = []
    matched_vehicles[vsr_config["VSR"]] = []
for vehicle_name, vehicle_params in vehicle_configuration.items():
    match_found = False
    for vsr_config in release_config:
        if (
            vehicle_params['vehicle_sec'] == vsr_config['vehicle_sec'] and
            vehicle_params['vehicle_vdrive'] == vsr_config['vdrive'] and
            vehicle_params['vehicle_reecu'] == vsr_config['reecu'] and
            vehicle_params['depb_left'] == vsr_config['depb']
        ):
            match_found = True
            matched_vehicles[vsr_config["VSR"]].append(vehicle_name)
            break
    if match_found == False:
        for vsr_config in release_config:
            mismatched_vehicles[vsr_config["VSR"]].append({
                'name': vehicle_name,
                'params': {vehicle_key: vehicle_params[vehicle_key] for vsr_key, vehicle_key in zip(vsr_keys, vehicle_keys) if vehicle_params[vehicle_key] != vsr_config[vsr_key]}
            })

print("Mismatched Vehicles:")
print(mismatched_vehicles)
print("Matched Vehicles:")
print(matched_vehicles)

Mismatched Vehicles:
{'2.9.0': [{'name': 've-0008', 'params': {'vehicle_vdrive': '53.0.3', 'vehicle_reecu': '8.1.39761'}}, {'name': 've-0009', 'params': {'vehicle_vdrive': '53.0.3', 'vehicle_reecu': '8.1.23786'}}, {'name': 've-0011', 'params': {'vehicle_vdrive': '54.0.3', 'vehicle_reecu': '8.1.2'}}, {'name': 've-0014', 'params': {'vehicle_vdrive': '53.0.3', 'vehicle_reecu': '0.0.0', 'vehicle_sec': '0.0.0', 'depb_left': '2.1.1'}}, {'name': 've-0015', 'params': {'vehicle_reecu': '8.0.1'}}, {'name': 've-0020', 'params': {'vehicle_vdrive': '53.0.3', 'vehicle_reecu': '8.1.15620', 'depb_left': '0.0.0'}}, {'name': 've-us-0001', 'params': {'vehicle_vdrive': '53.0.5', 'vehicle_reecu': '8.1.2'}}, {'name': 've-us-0002', 'params': {'vehicle_vdrive': '53.0.5', 'vehicle_reecu': '8.1.2'}}], '2.10.2': [{'name': 've-0008', 'params': {'vehicle_vdrive': '53.0.3', 'vehicle_reecu': '8.1.39761'}}, {'name': 've-0009', 'params': {'vehicle_vdrive': '53.0.3', 'vehicle_reecu': '8.1.23786'}}, {'name': 've-0011', 

In [50]:
mismatched_vehicles_string = ""
for release in mismatched_vehicles:
    mismatched_vehicles_string = mismatched_vehicles_string + "*" + release + "*" + "\n"
    for vehicle in mismatched_vehicles[release]:
        mismatched_vehicles_string = mismatched_vehicles_string + vehicle["name"] + "\n"
        for key in vehicle["params"]:
            mismatched_vehicles_string = mismatched_vehicles_string + " " + key + " " + vehicle["params"][key] + "\n"
        mismatched_vehicles_string = mismatched_vehicles_string + "\n"

print(mismatched_vehicles_string)

*2.9.0*
ve-0008
 vehicle_vdrive 53.0.3
 vehicle_reecu 8.1.39761

ve-0009
 vehicle_vdrive 53.0.3
 vehicle_reecu 8.1.23786

ve-0011
 vehicle_vdrive 54.0.3
 vehicle_reecu 8.1.2

ve-0014
 vehicle_vdrive 53.0.3
 vehicle_reecu 0.0.0
 vehicle_sec 0.0.0
 depb_left 2.1.1

ve-0015
 vehicle_reecu 8.0.1

ve-0020
 vehicle_vdrive 53.0.3
 vehicle_reecu 8.1.15620
 depb_left 0.0.0

ve-us-0001
 vehicle_vdrive 53.0.5
 vehicle_reecu 8.1.2

ve-us-0002
 vehicle_vdrive 53.0.5
 vehicle_reecu 8.1.2

*2.10.2*
ve-0008
 vehicle_vdrive 53.0.3
 vehicle_reecu 8.1.39761

ve-0009
 vehicle_vdrive 53.0.3
 vehicle_reecu 8.1.23786

ve-0011
 vehicle_vdrive 54.0.3

ve-0014
 vehicle_vdrive 53.0.3
 vehicle_reecu 0.0.0
 vehicle_sec 0.0.0
 depb_left 2.1.1

ve-0015
 vehicle_vdrive 52.0.13
 vehicle_reecu 8.0.1

ve-0020
 vehicle_vdrive 53.0.3
 vehicle_reecu 8.1.15620
 depb_left 0.0.0

ve-us-0001
 vehicle_vdrive 53.0.5

ve-us-0002
 vehicle_vdrive 53.0.5




In [42]:
mismatched_telestations = {}
matched_telestations = {}
vsr_keys = ["vdrive", "reecu", "telestation_sec"]
telestation_keys = ["telestation_vdrive", "telestation_reecu", "telestation_sec"]

for vsr_config in release_config:
    mismatched_telestations[vsr_config["VSR"]] = []
    matched_telestations[vsr_config["VSR"]] = []
    
for telestation_name, telestation_params in telestation_configuration.items():
    match_found = False
    for vsr_config in release_config:
        if (
            telestation_params['telestation_sec'] == vsr_config['telestation_sec'] and
            telestation_params['telestation_vdrive'] == vsr_config['vdrive'] and
            telestation_params['telestation_reecu'] == vsr_config['reecu']
        ):
            match_found = True
            matched_telestations[vsr_config["VSR"]].append(telestation_name)
            break
    if match_found == False:
        for vsr_config in release_config:
            mismatched_telestations[vsr_config["VSR"]].append({
                'name': telestation_name,
                'params': {telestation_key: telestation_params[telestation_key] for vsr_key, telestation_key in zip(vsr_keys, telestation_keys) if telestation_params[telestation_key] != vsr_config[vsr_key]}
            })

print("Mismatched Telestations:")
print(mismatched_telestations)
print("Matched Telestations:")
print(matched_telestations)

Mismatched Telestations:
{'2.9.0': [{'telestation_name': 'ts-ber-0009', 'params': {'telestation_vdrive': '54.0.3', 'telestation_reecu': '8.1.2'}}, {'telestation_name': 'ts-ber-0010', 'params': {'telestation_vdrive': '54.0.3'}}, {'telestation_name': 'ts-gd-ber-0001', 'params': {'telestation_vdrive': '53.0.3', 'telestation_reecu': '0.0.0', 'telestation_sec': '0.0.0'}}, {'telestation_name': 'ts-ber-0005', 'params': {'telestation_reecu': '8.0.1'}}, {'telestation_name': 'ts-ber-0013', 'params': {'telestation_vdrive': '52.0.11', 'telestation_reecu': '8.0.1'}}, {'telestation_name': 'ts-ber-0004', 'params': {'telestation_vdrive': '50.0.11', 'telestation_reecu': '6.9.51106'}}, {'telestation_name': 'ts-ber-0002', 'params': {'telestation_vdrive': '53.0.3', 'telestation_reecu': '8.1.2'}}, {'telestation_name': 'ts-las-0001', 'params': {'telestation_vdrive': '53.0.5', 'telestation_reecu': '8.1.2'}}, {'telestation_name': 'ts-las-0002', 'params': {'telestation_vdrive': '53.0.5', 'telestation_reecu': '

In [ ]:
def misconfiguration_string(mismatched_configuraiton: dict, ):
    mismatched_vehicles_string = ""
    for release in mismatched_configuraiton:
        mismatched_vehicles_string = mismatched_vehicles_string + "*" + release + "*" + "\n"
        for system in mismatched_configuraiton[release]:
            mismatched_vehicles_string = mismatched_vehicles_string + system["name"] + "\n"
            for key in system["params"]:
                mismatched_vehicles_string = mismatched_vehicles_string + " " + key + " " + system["params"][key] + "\n"
            mismatched_vehicles_string = mismatched_vehicles_string + "\n"

    return mismatched_vehicles_string